In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. DATA LOADING & PREPROCESSING
df = pd.read_csv('car_dataset.csv')
print("Dataset shape:", df.shape)

Dataset shape: (2095, 14)


In [ ]:
# Clean data
df.drop('Unnamed: 0', axis=1, inplace=True)


In [ ]:
# Feature Engineering
df['age'] = 2025 - df['year']
df['price_per_km'] = df['selling_price'] / (df['km_driven'] + 1)
df['power_per_seat'] = df['max_power (in bph)'] / df['seats']
df.drop('year', axis=1, inplace=True)

In [ ]:
# FIXED: Save original categories BEFORE encoding
categorical_cols = ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'Mileage Unit']
label_encoders = {}
all_categories = {}  # Store all possible categories

In [ ]:
for col in categorical_cols:
    # Get all unique values from training data
    all_categories[col] = sorted(df[col].unique())
    le = LabelEncoder()
    le.fit(all_categories[col])  # Fit on all known categories
    df[col] = le.transform(df[col])
    label_encoders[col] = le

In [ ]:
# Features & Target
feature_cols = ['km_driven', 'fuel', 'seller_type', 'transmission', 'owner',
                'seats', 'max_power (in bph)', 'Mileage Unit', 'Mileage',
                'Engine (CC)', 'name', 'age', 'price_per_km', 'power_per_seat']

In [ ]:
X = df[feature_cols]
y = df['selling_price']

In [ ]:
print(f"Features shape: {X.shape}")

Features shape: (2095, 14)


In [ ]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 2. RANDOM FOREST MODEL
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

RandomForestRegressor(max_depth=15, min_samples_leaf=5, min_samples_split=10,
                      n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
# 3. EVALUATION
train_pred = model.predict(X_train_scaled)
test_pred = model.predict(X_test_scaled)

train_mae = mean_absolute_error(y_train, train_pred)
test_mae = mean_absolute_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)

print(f"Train MAE: ₹{train_mae:,.0f}")
print(f"Test MAE:  ₹{test_mae:,.0f}")
print(f"Test R²:   {test_r2:.4f}")

Train MAE: ₹28,040
Test MAE:  ₹32,629
Test R²:   0.9487


In [ ]:
# 4. SAVE EVERYTHING
joblib.dump(model, 'car_price_rf_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(feature_cols, 'feature_columns.pkl')
joblib.dump(all_categories, 'all_categories.pkl')  # NEW: Save categories

print("saved")

saved


In [ ]:
# 5. FIXED PREDICTION FUNCTION - Handles Unseen Labels
def predict_car_price(car_data):
    """ FIXED: Handles unseen labels like 'Maruti' perfectly"""
    try:
        # Load everything
        model = joblib.load('car_price_rf_model.pkl')
        scaler = joblib.load('scaler.pkl')
        label_encoders = joblib.load('label_encoders.pkl')
        feature_cols = joblib.load('feature_columns.pkl')
        all_categories = joblib.load('all_categories.pkl')

        # Prepare data
        df_pred = pd.DataFrame([car_data])

        # Feature engineering
        if 'year' in df_pred.columns:
            df_pred['age'] = 2025 - df_pred['year'].astype(int)
        else:
            df_pred['age'] = 5

        df_pred['price_per_km'] = 1.0
        df_pred['power_per_seat'] = df_pred['max_power (in bph)'].iloc[0] / df_pred['seats'].iloc[0]

        # FIXED: Safe encoding for unseen categories
        for col in categorical_cols:
            if col in df_pred.columns:
                value = df_pred[col].iloc[0]
                # If value not in encoder classes, use most common (index 0)
                if str(value) not in label_encoders[col].classes_:
                    # Find closest match or use first category
                    df_pred[col] = 0  # Most common category
                    print(f" Unknown '{col}': '{value}' → using common category (0)")
                else:
                    df_pred[col] = label_encoders[col].transform([str(value)])[0]

        # Select features in exact order
        X_pred = df_pred[feature_cols].fillna(0)
        X_pred_scaled = scaler.transform(X_pred)

        # Predict
        prediction = model.predict(X_pred_scaled)[0]

        return {
            'predicted_price': f"₹{prediction:,.0f}"
        }

    except Exception as e:
        return {'error': str(e)}


In [ ]:
# 6. TEST WITH 'Maruti' - NOW WORKS!
sample_car = {
    'name': 'Maruti',
    'km_driven': 50000,
    'fuel': 'Petrol',
    'seller_type': 'Individual',
    'transmission': 'Manual',
    'owner': 'First Owner',
    'seats': 5,
    'max_power (in bph)': 74.0,
    'Mileage Unit': 'kmpl',
    'Mileage': 20.0,
    'Engine (CC)': 1248,
    'year': 2018
}



result = predict_car_price(sample_car)
print("Sample Prediction:", result)


Sample Prediction: {'predicted_price': '₹283,503'}
